<a href="https://colab.research.google.com/github/PlumNoseBear/PAINN/blob/main/%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_%D1%86%D0%B5%D0%BD%D1%8B_%D0%BA%D1%80%D0%B8%D0%BF%D1%82%D0%BE_%D0%B2%D0%B0%D0%BB%D1%8E%D1%82_%D1%81_%D0%91%D0%B8%D0%BD%D0%B0%D0%BD%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Тестовое задание
Программа на Python, которая в реальном времени следит за ценой криптовалюты на Binance, программа выводит сообщение в консоль.

In [ ]:
# @title Импорт бибилиотек
import aiohttp
import asyncio
from typing import Dict, Optional
from datetime import datetime
from zoneinfo import ZoneInfo
from IPython.display import display, HTML

In [ ]:
# @title Определние функций
BASE_URL = "https://data-api.binance.vision"
async def get_prices(*symbols):
    url = f"{BASE_URL}/api/v3/ticker/price"

    async with aiohttp.ClientSession() as session:
        if not symbols:
            try:
                async with session.get(url) as resp:
                    resp.raise_for_status()
                    data = await resp.json()
                    return {item["symbol"]: float(item["price"]) for item in data}
            except Exception as e:
                print(f"Ошибка при получении всех цен: {e}")
                return {}

        tasks = []
        for symbol in symbols:
            tasks.append(fetch_single_price(session, url, symbol))

        results = await asyncio.gather(*tasks, return_exceptions=True)
        return {sym: price for sym, price in results if price is not None}

async def fetch_single_price(session, base_url, symbol):
    try:
        async with session.get(base_url, params={"symbol": symbol}) as resp:
            if resp.status == 200:
                data = await resp.json()
                return data["symbol"], float(data["price"])
            else:
                text = await resp.text()
                print(f"{symbol}: HTTP {resp.status} — {text[:200]}")
                return symbol, None
    except Exception as e:
        print(f"Ошибка при получении {symbol}: {e}")
        return symbol, None

async def main(symbols: list, sleep_time: float = 5.0, timezone: str = 'Asia/Yekaterinburg'):
    tz = ZoneInfo(timezone)
    previous_prices: Dict[str, Optional[float]] = {sym: None for sym in symbols}

    try:
        while True:
            current_prices = await get_prices(*symbols)

            # Формируем строку с текущим временем и ценами
            now = datetime.now(tz)
            time_str = now.strftime("%d.%m.%Y %H:%M:%S")
            parts = [f"{time_str} :"]

            for symbol in symbols:
                current = current_prices.get(symbol)
                previous = previous_prices[symbol]

                if current is not None:
                    parts.append(format_price_change_html(current, previous, symbol))
                    previous_prices[symbol] = current
                else:
                    parts.append(f"{symbol}:  Не удалось получить цену")

            # Выводим всё в одной строке как HTML
            output_str = "   ".join(parts)
            display(HTML(output_str))

            await asyncio.sleep(sleep_time)
    except KeyboardInterrupt:
        print(" Завершено пользователем.")



In [ ]:
# @title Добавка цвета тикерам (на HTML)
def format_price_change_html(current: float, previous: Optional[float], symbol: str) -> str:
    if previous is None:
        return f"{symbol}: ${current:,.2f} !"

    # Определяем состояние: рост, падение или без изменений
    state = 'up' if current > previous else 'down' if current < previous else 'same'

    # Используем match для определения цвета и стрелки
    match state:
        case 'up':
            arrow = "↑"
            color = 'green'
        case 'down':
            arrow = "↓"
            color = 'red'
        case 'same':
            arrow = "!"
            color = 'gray'
        case _:  # на случай, если что-то пошло не так
            arrow = " ? "
            color = 'gray'

    return f"{symbol}: <span style='color:{color}'>${current:,.2f} {arrow}</span>"

In [ ]:
# @title Форма с  чекбоксами выбора тикеров

# Определяем параметры формы
BTCUSDT = True  # @param {type:"boolean"}
ETHUSDT = True  # @param {type:"boolean"}
XRPUSDT = False  # @param {type:"boolean"}
BNBUSDT = False  # @param {type:"boolean"}
SOLUSDT = False  # @param {type:"boolean"}
ADAUSDT = True # @param {type:"boolean"}
DOTUSDT = True  # @param {type:"boolean"}
# Функция для обработки выбранных опций
# Указываем, какие переменные отслеживать:
COIN_VARS = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BNBUSDT','SOLUSDT', 'ADAUSDT', 'DOTUSDT']

def get_selected_coins():
    selected = []
    # Берём значения из глобальной области (в Colab ячейка — это globals())
    for name in COIN_VARS:
        value = globals().get(name)
        match value:
            case True:
                selected.append(name)
            # case False | None:  # можно группировать, но не обязательно
            case _:
                pass
    return selected
symbols=get_selected_coins()


In [ ]:
# @title Проверка выбраных тикеров
print(symbols)

['BTCUSDT', 'ETHUSDT', 'ADAUSDT', 'DOTUSDT']


In [ ]:
# @title РУЧНОЕ ЗАДАНИЕ СПИСКА
symbols=["ETHUSDT", "BTCUSDT", "SOLUSDT","NEOUSDT"]

In [ ]:
await main(symbols, sleep_time = 1)

CancelledError: 